In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn import preprocessing

import gradient
import dataset
import computations
import layer
#from layer import Linear, Softmax, Gradient
import network

import datetime
import time

In [2]:
np.random.seed(400)
np.seterr(over='raise');
plt.rcParams['figure.figsize'] = (15.0, 5.0)

In [3]:
network1 = network.Network()
cifar = dataset.CIFAR_IMAGES()
#asgn1.test_batch_images(cifar_batch1)

In [4]:
#### Exercise - 4 ###
# Read in the data & initialize the parameters of the network
filePathLocal_labels = 'Dataset/batches.meta'
#filePathLocal_data_TRAIN = 'Dataset/data_batch_1'
#filePathLocal_data_VALIDATION = 'Dataset/data_batch_2'
filePathLocal_data_ALL = ['Dataset/data_batch_1', 'Dataset/data_batch_2', 'Dataset/data_batch_3', 'Dataset/data_batch_4', 'Dataset/data_batch_5']
filePathLocal_data_TEST = 'Dataset/test_batch'

filePathList = (filePathLocal_data_ALL, filePathLocal_data_TEST)

# Read TRAIN, VALIDATION, TEST data into numpy arrays (numpy.ndarray) from local files
network1.ReadData_Exercise4(cifar, filePathList)
# X = (d, N), Y = (K, N), y = (N,)   # N=number of total images in X
# X = (3072, 10000), Y = (10, 10000), y = (10000,)

# Find the MEAN and STD of trainX and broadcast them for matrix calculations
# trainX_Broadcast_MeanStd = (mean_train_X_broadcast, std_train_X_broadcast)
#trainX_Broadcast_MeanStd = network1.MeanStd_Train_X(network1.train_X)

# MEL
# Transform the INPUT to have zero mean ** Check that one if we need to transfer all of them separately or
# only having the normalization as in here??
# Normalize all INPUT data by using MEAN and STD of TRAIN DATA
'''
train_X_Norm = network1.NormalizeData(network1.train_X, trainX_Broadcast_MeanStd)
validation_X_Norm = network1.NormalizeData(network1.validation_X, trainX_Broadcast_MeanStd)
test_X_Norm = network1.NormalizeData(network1.test_X, trainX_Broadcast_MeanStd)
'''

train_X_Norm = network1.NormalizeData_Broadcast(network1.train_X, network1.train_X)
validation_X_Norm = network1.NormalizeData_Broadcast(network1.validation_X, network1.train_X)
test_X_Norm = network1.NormalizeData_Broadcast(network1.test_X, network1.train_X)

# mu = 0; d = network1.train_X.shape[0]; m = 50; K = network1.train_Y.shape[0]
# we will use only 20 of 3072 to have a dimension reduction in comparing grad_analytic and grad_Numerical
#mu = 0; d = 20; m = 50; K = network1.train_Y.shape[0]
mu = 0; d = network1.train_X.shape[0]; m = 50; K = network1.train_Y.shape[0]

initial_sizes = (mu, d, m, K)

sigma1 = 1 / int(np.sqrt(d))
sigma2 = 1 / np.sqrt(m)

# Generate W1, W2, b1, b2 matrices with initial values
#(W1, W2, b1, b2) = network1.Initialize_W_b(d, m, K, sigma1, sigma2)
(W1, W2, b1, b2) = network1.Initialize_W_b(initial_sizes, sigma1, sigma2)
# W1 = (50, 3072), W2 = (10, 50), b1 = (50, 1), b2 = (10, 1) # if we use the whole dimensions/features
# W1 = (50, 20), W2 = (10, 50), b1 = (50, 1), b2 = (10, 1) # if we use 20 dimensions/features
# W1 = (m, d), W2 = (K, m), b1 = (m, 1), b2 = (K, 1)

In [ ]:
trainX_Broadcast_MeanStd[0].shape

In [ ]:
network1.validation_X.shape

In [ ]:
network1.train_X.mean(axis=1).shape

In [5]:
'''
print("Mean-STD:\n trainX_Broadcast_MeanStd[0]=\n{}\n\ntrainX_Broadcast_MeanStd[1]=\n{}".\
      format(trainX_Broadcast_MeanStd[0], trainX_Broadcast_MeanStd[1]))
print("\n\nMean-STD:\n train_X_Norm.mean(axis=1)={}\n\n".format(train_X_Norm.mean(axis=1)))
'''
print("Shape check:\n train_X_Norm={}\t validation_X_Norm={}\t test_X_Norm={}".format(train_X_Norm.shape, validation_X_Norm.shape, test_X_Norm.shape))
print(" train_Y={}\t\t validation_Y={}\t\t test_Y={}".format(network1.train_Y.shape, network1.validation_Y.shape, network1.test_Y.shape))
print(" train_y={}\t\t validation_y={}\t\t\t test_y={}".format(network1.train_y.shape, network1.validation_y.shape, network1.test_y.shape))
print(" W1={}\t\t\t W2={}\t\t\t\t b1={}\t\t\t b2={}".format(W1.shape, W2.shape, b1.shape, b2.shape))

Shape check:
 train_X_Norm=(3072, 45000)	 validation_X_Norm=(3072, 5000)	 test_X_Norm=(3072, 10000)
 train_Y=(10, 45000)		 validation_Y=(10, 5000)		 test_Y=(10, 10000)
 train_y=(45000,)		 validation_y=(5000,)			 test_y=(10000,)
 W1=(50, 3072)			 W2=(10, 50)				 b1=(50, 1)			 b2=(10, 1)


In [ ]:
type(network1.validation_X)

In [ ]:
#X_batch #S1 #W1 #W2 #H #S #P  #Y_batch.shape
#print(" S1={}\t\t\t H={}\t\t\t\t S={}\t\t\t P={}".format(S1.shape, H.shape, S.shape, P.shape))

In [ ]:
#exp_s = np.exp(S)
#print(exp_s)

In [6]:
#def Plot_Train_Validation_Cost_Accurracy(self, Cost_Train, Cost_Validation, Acc_Train, Acc_Validation):
def Plot_Train_Validation_Cost_Accurracy(Cost_Train, Cost_Validation, Acc_Train, Acc_Validation):
        plt.subplot(1,2,1)
        plt.plot(Cost_Train, 'g-', label='Train')
        plt.plot(Cost_Validation, 'r-', label='Validation')
        plt.title('Cost Comparison')
        plt.xlabel('Step*10')
        plt.ylabel('Cost')
        plt.legend()
        plt.grid('on')
        
        plt.subplot(1,2,2)
        plt.plot(Acc_Train, 'g-', label='Train')
        plt.plot(Acc_Validation, 'r-', label='Validation')
        plt.title('Accuracy Comparison')
        plt.xlabel('Step*10')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid('on')

In [26]:
def Coarse_Search(l_min, l_max):
    l = l_min + (l_max - l_min) * np.random.uniform(0,1)
    lambda_coarse = pow(10, l)
    return lambda_coarse

In [29]:
def Train_Cyclical_Coarse(network1, train_X_Norm, validation_X_Norm, n_cycles):
    '''
    Up until now, we have trained our networks with Vanilla mini-batch gradient descent. 
    To help speed up training times and avoid time-consuming searches for good values of eta, we will now
    implement mini-batch-GD training where the learning rate at each update step is defined in a cylical way
    check equations (14) and (15) in the assignment.
    '''
    #n_epocs = 200      # number of times we will iterate on the entire data (10K images in our case-1 batch)
    n_batch = 100       # the size of the mini-batch. in other words, number of images in 1 mini-batch.
    #eta = 0.001         # learning rate (step-size)
    lambda_cost = 0.01  # regularization coefficient (punishment)
    d = train_X_Norm.shape[0]
    m = 50 
    K = network1.train_Y.shape[0]  # number of classes
    eta_min = 1e-5
    #eta_max = 1e-1  
    eta_max = 2e-2  # MEL: If I use an eta bigger than 2e-2, I receive an overflow error for SOFTMAX 
    #n_steps = 500                  # number of steps in half a cycle (from eta_min to eta_max) 

    # Generate W1, W2, b1, b2 matrices with initial values
    # (W1, W2, b1, b2) = network1.Initialize_W_b(d, m, K)
    sigma1 = 1 / int(np.sqrt(d))
    sigma2 = 1 / np.sqrt(m)
    #(W1, W2, b1, b2) = network1.Initialize_W_b(d, m, K, sigma1, sigma2)
    initial_sizes = [0, d, m, K]
    (W1, W2, b1, b2) = network1.Initialize_W_b(initial_sizes, sigma1, sigma2)

    W = [W1, W2]
    b = [b1, b2]

    linearLayer1 = layer.Linear()
    reluLayer = layer.ReLU()             # not an exact layer but operational step..
    linearLayer2 = layer.Linear()
    softmaxLayer = layer.Softmax()       # not an exact layer but operational step..
    ReLUlayer = layer.ReLU()
    grad = gradient.Gradient()

    layers = [linearLayer1, reluLayer, linearLayer2, softmaxLayer]
    # total_batch = how many mini-batches will we need to cover the entire training set?
    # we will use 45000 images for training = train_X_Norm.shape[1]
    # total_batch = 45000/100 = 450
    total_batch = int(np.floor(train_X_Norm.shape[1] / n_batch))
    
    # A full cycle once go up (from eta_min to eta_max) and once go down (from eta_max to eta_min)
    # so we multiply by 2
    #n_cycles = 1   # corresponds to "L array" in 2.L.ns in the assignment .. 
    # L=the current cycle number ... n_cycles=the total number of cycles to be applied
    # so L is an element of n_cycle >> i.e. L = {0, 1, 2, 3} if n_cycles = 4 
    # n_epocs=10 for 1 full cycle having n_steps=500 and n_batch=100
    n_steps = 2 * total_batch
    # n_steps / total_batch = 2 is picked before this value was used as k in exercise-3
    # but in exercise-3, we were given the n_steps, now it is calculated and k is given as 2
    # so n_epocs = 2 * 2 * 2 = 8 in our case
    n_epocs = int(2 * n_cycles * (n_steps / total_batch))
    #n_sanity_batch = 1
    # we will only use 100 images for sanity check that means there will be only 1 mini-batch
    #n_sanity_batch = 1
    #n_test_images = n_batch * n_sanity_batch
    
    #J_epocs_train = np.zeros(n_epocs)         # cost array     - will keep costs per epoc (iteration)
    #Accuracy_epocs_train = np.zeros(n_epocs)  # accuracy array - will keep accuracy per epoc (iteration)
    
    n_records = int(2 * n_cycles * n_steps / 10)
    #J_epocs_train = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    #Accuracy_epocs_train = np.zeros(n_records) # we will only record once in 10 iteration
    '''
    J_epocs_train = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_train = np.zeros(n_records) # we will only record once in 10 iteration
    LambdaCost_train = np.zeros(n_records)
    eta_train = np.zeros(n_records)
    
    J_epocs_validation = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_validation = np.zeros(n_records) # we will only record once in 10 iteration
    '''

    J_epocs_train = np.zeros(n_epocs) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_train = np.zeros(n_epocs) # we will only record once in 10 iteration
    LambdaCost_train = np.zeros(n_epocs)
    eta_train = np.zeros(n_epocs)
    
    J_epocs_validation = np.zeros(n_epocs) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_validation = np.zeros(n_epocs) # we will only record once in 10 iteration

    # t = iteration number for eta (don't confuse with the epocs iteration!)
    t = 0
    # cycleID in use.. if n_cycles=0 then cycle max will be 0. else it will be incremented by 1 at each 2*n_steps
    cycle_no = 0
    n_records = 0
    list_params = []
    start_time = datetime.datetime.now()
    print('lambda_cost, eta, J_train, J_validation, A_train, A_validation')
    for e in range(n_epocs):
        lambda_cost = Coarse_Search(-5, -1)
        for batch in range(total_batch):
            #print('cycle_no: {}, e: {}, batch: {}, t: {}, eta: {}'.format(cycle_no, e, batch, t, eta))
            index_list = list(range(batch * n_batch, (batch + 1) * n_batch))
            # shuffling is not necessary but good to have
            np.random.shuffle(index_list)
            #X_batch = train_X_Norm[:, index_list]
            X_batch = train_X_Norm[:, index_list]
            Y_batch = network1.train_Y[:, index_list]

            P, H = network1.EvaluationClassifier(layers, X_batch, W, b)

            G2 = -np.subtract(network1.train_Y[:, index_list], P)  # G
            N2 = Y_batch.shape[1] #N

            # N, G
            (grad_W2, grad_b2, G1) = grad.ComputeGradients_Linear_HiddenLayer(N2, G2, H, lambda_cost, W2) 
            G0 = reluLayer.Backward(G1, H)
            N1 = H.shape[1] #N
            # N, G
            (grad_W1, grad_b1) = grad.ComputeGradients_Linear_FirstLayer(N1, G0, X_batch, lambda_cost, W1)
            
            if 2*cycle_no*n_steps <= t and t <= (2*cycle_no+1)*n_steps:
                eta = eta_min + (t - 2*cycle_no*n_steps)/n_steps*(eta_max-eta_min)
            elif (2*cycle_no+1)*n_steps <= t and t <= 2*(cycle_no+1)*n_steps:
                eta = eta_max - (t - (2*cycle_no+1)*n_steps)/n_steps*(eta_max-eta_min)
            
            # W1star, W2star
            W1 = W1 - eta * grad_W1
            W2 = W2 - eta * grad_W2

            # b1star, b2star
            bstar_m1 = b1 - eta * grad_b1
            b1 = bstar_m1[:, :1]  # there's a broadcast issue needs to be fixed, that's why we pick only 1 column

            bstar_m2 = b2 - eta * grad_b2
            b2 = bstar_m2[:, :1]  # broadcast issue, this is a quick workaround - use 1 column
            
            #print('cycle_no: {}, e: {}, batch: {}, t: {}, eta: {}'.format(cycle_no, e, batch, t, eta))
            
            '''
            # 12:43 if I calculate J for each t
            if t % 10 == 9:    
                W = [W1, W2]
                b = [b1, b2]
                            
                eta_train[n_records] = eta
                LambdaCost_train[n_records] = lambda_cost

                print('\nepoch: {}, batch: {}, n_records: {}, t: {}'.format(e, batch, n_records, t))
                print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_train'))
                J_train = network1.Cost(train_X_Norm, network1.train_Y, W, b, lambda_cost)

                J_epocs_train[n_records] = J_train

                P, H = network1.EvaluationClassifier(layers, train_X_Norm, W, b)
                k_train = np.argmax(P, axis=0)

                A_train = network1.ComputeAccuracy(k_train, network1.train_y)
                Accuracy_epocs_train[n_records] = A_train

                print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_validation'))
                J_validation = network1.Cost(validation_X_Norm , network1.validation_Y, W, b, lambda_cost)

                J_epocs_validation[n_records] = J_validation

                P_val, H_val = network1.EvaluationClassifier(layers, validation_X_Norm, W, b)
                k_validation = np.argmax(P_val, axis=0)

                A_validation = network1.ComputeAccuracy(k_validation, network1.validation_y)
                Accuracy_epocs_validation[n_records] = A_validation
                
                list_temp = [lambda_cost, eta, J_train, J_validation, A_train, A_validation]
                print('params: {} ... time: {}'.format(list_temp, datetime.datetime.now()))
                list_params.append(list_temp)                
                n_records += 1
            '''
            t += 1
            
            if t % (2 * n_steps) == 0:
                cycle_no += 1

        W = [W1, W2]
        b = [b1, b2]

        eta_train[e] = eta
        LambdaCost_train[e] = lambda_cost

        print('\nepoch: {}, batch: {}, t: {}'.format(e, batch, t))
        print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_train'))
        J_train = network1.Cost(train_X_Norm, network1.train_Y, W, b, lambda_cost)

        J_epocs_train[e] = J_train

        P, H = network1.EvaluationClassifier(layers, train_X_Norm, W, b)
        k_train = np.argmax(P, axis=0)

        A_train = network1.ComputeAccuracy(k_train, network1.train_y)
        Accuracy_epocs_train[e] = A_train

        print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_validation'))
        J_validation = network1.Cost(validation_X_Norm , network1.validation_Y, W, b, lambda_cost)

        J_epocs_validation[e] = J_validation

        P_val, H_val = network1.EvaluationClassifier(layers, validation_X_Norm, W, b)
        k_validation = np.argmax(P_val, axis=0)

        A_validation = network1.ComputeAccuracy(k_validation, network1.validation_y)
        Accuracy_epocs_validation[e] = A_validation
                
        list_temp = [lambda_cost, eta, J_train, J_validation, A_train, A_validation]
        print('params: {} ... time: {}'.format(list_temp, datetime.datetime.now()))
        list_params.append(list_temp)                
        #n_records += 1
        

    end_time = datetime.datetime.now()
    
    print("Calculation time of Train_Cyclical: " + str(end_time - start_time))
    print("x-axis (steps) must be multiplied by 10 since the values recorded once in every 10 Cycle")
        
    #print('J_epocs_train: {}'.format(J_epocs_train))
    #print('Accuracy_epocs_train: {}'.format(Accuracy_epocs_train))
    
    Plot_Train_Validation_Cost_Accurracy(J_epocs_train, J_epocs_validation, Accuracy_epocs_train, Accuracy_epocs_validation)
    
    
    

In [30]:
Train_Cyclical_Coarse(network1, train_X_Norm, validation_X_Norm, 1)

lambda_cost, eta, J_train, J_validation, A_train, A_validation

epoch: 0, batch: 449, t: 450
time: 2020-05-06 11:20:46.378931 ... operation: J_train
time: 2020-05-06 11:24:11.379949 ... operation: J_validation
params: [1.0175839366222833e-05, 0.009982788888888889, 26.907630799519847, 26.51228011947062, 0.2132888888888889, 0.2166] ... time: 2020-05-06 11:24:11.880633

epoch: 1, batch: 449, t: 900
time: 2020-05-06 11:24:12.617747 ... operation: J_train


/Users/melih/Documents/Masters/KTH_DASE-ICT Innovation_Data Science/Deep Learning/Assignment-2/_codeMel/network.py:206: RuntimeWarning: invalid value encountered in true_divide
  P = exp_s / np.sum(exp_s, axis=0)
/Users/melih/Documents/Masters/KTH_DASE-ICT Innovation_Data Science/Deep Learning/Assignment-2/_codeMel/network.py:208: RuntimeWarning: divide by zero encountered in log
  cross_entropy_loss = -np.log(np.dot(Y.transpose(), P))
/Users/melih/Documents/Masters/KTH_DASE-ICT Innovation_Data Science/Deep Learning/Assignment-2/_codeMel/layer.py:92: RuntimeWarning: invalid value encountered in true_divide
  p = exp_s / np.sum(exp_s, axis=0)


time: 2020-05-06 11:25:16.266745 ... operation: J_validation
params: [0.043433826635473606, 0.01997778888888889, nan, nan, 0.06722222222222222, 0.062] ... time: 2020-05-06 11:25:16.934901


FloatingPointError: overflow encountered in exp

In [41]:
def Train_Cyclical_Coarse_lambda(network1, train_X_Norm, validation_X_Norm, n_cycles, lambda_coarse):
    '''
    Up until now, we have trained our networks with Vanilla mini-batch gradient descent. 
    To help speed up training times and avoid time-consuming searches for good values of eta, we will now
    implement mini-batch-GD training where the learning rate at each update step is defined in a cylical way
    check equations (14) and (15) in the assignment.
    '''
    #n_epocs = 200      # number of times we will iterate on the entire data (10K images in our case-1 batch)
    n_batch = 100       # the size of the mini-batch. in other words, number of images in 1 mini-batch.
    #eta = 0.001         # learning rate (step-size)
    lambda_cost = 0.01  # regularization coefficient (punishment)
    d = train_X_Norm.shape[0]
    m = 50 
    K = network1.train_Y.shape[0]  # number of classes
    eta_min = 1e-5
    #eta_max = 1e-1  
    eta_max = 2e-4  # MEL: If I use an eta bigger than 2e-2, I receive an overflow error for SOFTMAX 
    #n_steps = 500                  # number of steps in half a cycle (from eta_min to eta_max) 

    # Generate W1, W2, b1, b2 matrices with initial values
    # (W1, W2, b1, b2) = network1.Initialize_W_b(d, m, K)
    sigma1 = 1 / int(np.sqrt(d))
    sigma2 = 1 / np.sqrt(m)
    #(W1, W2, b1, b2) = network1.Initialize_W_b(d, m, K, sigma1, sigma2)
    initial_sizes = [0, d, m, K]
    (W1, W2, b1, b2) = network1.Initialize_W_b(initial_sizes, sigma1, sigma2)

    W = [W1, W2]
    b = [b1, b2]

    linearLayer1 = layer.Linear()
    reluLayer = layer.ReLU()             # not an exact layer but operational step..
    linearLayer2 = layer.Linear()
    softmaxLayer = layer.Softmax()       # not an exact layer but operational step..
    ReLUlayer = layer.ReLU()
    grad = gradient.Gradient()

    layers = [linearLayer1, reluLayer, linearLayer2, softmaxLayer]
    # total_batch = how many mini-batches will we need to cover the entire training set?
    # we will use 45000 images for training = train_X_Norm.shape[1]
    # total_batch = 45000/100 = 450
    total_batch = int(np.floor(train_X_Norm.shape[1] / n_batch))
    
    # A full cycle once go up (from eta_min to eta_max) and once go down (from eta_max to eta_min)
    # so we multiply by 2
    #n_cycles = 1   # corresponds to "L array" in 2.L.ns in the assignment .. 
    # L=the current cycle number ... n_cycles=the total number of cycles to be applied
    # so L is an element of n_cycle >> i.e. L = {0, 1, 2, 3} if n_cycles = 4 
    # n_epocs=10 for 1 full cycle having n_steps=500 and n_batch=100
    n_steps = 2 * total_batch
    # n_steps / total_batch = 2 is picked before this value was used as k in exercise-3
    # but in exercise-3, we were given the n_steps, now it is calculated and k is given as 2
    # so n_epocs = 2 * 2 * 2 = 8 in our case
    n_epocs = int(2 * n_cycles * (n_steps / total_batch))
    #n_sanity_batch = 1
    # we will only use 100 images for sanity check that means there will be only 1 mini-batch
    #n_sanity_batch = 1
    #n_test_images = n_batch * n_sanity_batch
    
    #J_epocs_train = np.zeros(n_epocs)         # cost array     - will keep costs per epoc (iteration)
    #Accuracy_epocs_train = np.zeros(n_epocs)  # accuracy array - will keep accuracy per epoc (iteration)
    
    n_records = int(2 * n_cycles * n_steps / 10)
    #J_epocs_train = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    #Accuracy_epocs_train = np.zeros(n_records) # we will only record once in 10 iteration
    '''
    J_epocs_train = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_train = np.zeros(n_records) # we will only record once in 10 iteration
    LambdaCost_train = np.zeros(n_records)
    eta_train = np.zeros(n_records)
    
    J_epocs_validation = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_validation = np.zeros(n_records) # we will only record once in 10 iteration
    '''

    J_epocs_train = np.zeros(n_epocs) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_train = np.zeros(n_epocs) # we will only record once in 10 iteration
    LambdaCost_train = np.zeros(n_epocs)
    eta_train = np.zeros(n_epocs)
    
    J_epocs_validation = np.zeros(n_epocs) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_validation = np.zeros(n_epocs) # we will only record once in 10 iteration

    # t = iteration number for eta (don't confuse with the epocs iteration!)
    t = 0
    # cycleID in use.. if n_cycles=0 then cycle max will be 0. else it will be incremented by 1 at each 2*n_steps
    cycle_no = 0
    n_records = 0
    list_params = []
    start_time = datetime.datetime.now()
    print('lambda_cost, eta, J_train, J_validation, A_train, A_validation')
    lambda_cost = lambda_coarse
    for e in range(n_epocs):
        for batch in range(total_batch):
            #print('cycle_no: {}, e: {}, batch: {}, t: {}, eta: {}'.format(cycle_no, e, batch, t, eta))
            index_list = list(range(batch * n_batch, (batch + 1) * n_batch))
            # shuffling is not necessary but good to have
            np.random.shuffle(index_list)
            #X_batch = train_X_Norm[:, index_list]
            X_batch = train_X_Norm[:, index_list]
            Y_batch = network1.train_Y[:, index_list]

            P, H = network1.EvaluationClassifier(layers, X_batch, W, b)

            G2 = -np.subtract(network1.train_Y[:, index_list], P)  # G
            N2 = Y_batch.shape[1] #N

            # N, G
            (grad_W2, grad_b2, G1) = grad.ComputeGradients_Linear_HiddenLayer(N2, G2, H, lambda_cost, W2) 
            G0 = reluLayer.Backward(G1, H)
            N1 = H.shape[1] #N
            # N, G
            (grad_W1, grad_b1) = grad.ComputeGradients_Linear_FirstLayer(N1, G0, X_batch, lambda_cost, W1)
            
            if 2*cycle_no*n_steps <= t and t <= (2*cycle_no+1)*n_steps:
                eta = eta_min + (t - 2*cycle_no*n_steps)/n_steps*(eta_max-eta_min)
            elif (2*cycle_no+1)*n_steps <= t and t <= 2*(cycle_no+1)*n_steps:
                eta = eta_max - (t - (2*cycle_no+1)*n_steps)/n_steps*(eta_max-eta_min)
            
            eta = 1e-6
            # W1star, W2star
            W1 = W1 - eta * grad_W1
            W2 = W2 - eta * grad_W2

            # b1star, b2star
            bstar_m1 = b1 - eta * grad_b1
            b1 = bstar_m1[:, :1]  # there's a broadcast issue needs to be fixed, that's why we pick only 1 column

            bstar_m2 = b2 - eta * grad_b2
            b2 = bstar_m2[:, :1]  # broadcast issue, this is a quick workaround - use 1 column
            
            #print('cycle_no: {}, e: {}, batch: {}, t: {}, eta: {}'.format(cycle_no, e, batch, t, eta))
            
            '''
            # 12:43 if I calculate J for each t
            if t % 10 == 9:    
                W = [W1, W2]
                b = [b1, b2]
                            
                eta_train[n_records] = eta
                LambdaCost_train[n_records] = lambda_cost

                print('\nepoch: {}, batch: {}, n_records: {}, t: {}'.format(e, batch, n_records, t))
                print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_train'))
                J_train = network1.Cost(train_X_Norm, network1.train_Y, W, b, lambda_cost)

                J_epocs_train[n_records] = J_train

                P, H = network1.EvaluationClassifier(layers, train_X_Norm, W, b)
                k_train = np.argmax(P, axis=0)

                A_train = network1.ComputeAccuracy(k_train, network1.train_y)
                Accuracy_epocs_train[n_records] = A_train

                print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_validation'))
                J_validation = network1.Cost(validation_X_Norm , network1.validation_Y, W, b, lambda_cost)

                J_epocs_validation[n_records] = J_validation

                P_val, H_val = network1.EvaluationClassifier(layers, validation_X_Norm, W, b)
                k_validation = np.argmax(P_val, axis=0)

                A_validation = network1.ComputeAccuracy(k_validation, network1.validation_y)
                Accuracy_epocs_validation[n_records] = A_validation
                
                list_temp = [lambda_cost, eta, J_train, J_validation, A_train, A_validation]
                print('params: {} ... time: {}'.format(list_temp, datetime.datetime.now()))
                list_params.append(list_temp)                
                n_records += 1
            '''
            t += 1
            
            if t % (2 * n_steps) == 0:
                cycle_no += 1

        '''
        W = [W1, W2]
        b = [b1, b2]

        eta_train[e] = eta
        LambdaCost_train[e] = lambda_cost

        print('\nepoch: {}, batch: {}, t: {}'.format(e, batch, t))
        print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_train'))
        J_train = network1.Cost(train_X_Norm, network1.train_Y, W, b, lambda_cost)

        J_epocs_train[e] = J_train

        P, H = network1.EvaluationClassifier(layers, train_X_Norm, W, b)
        k_train = np.argmax(P, axis=0)

        A_train = network1.ComputeAccuracy(k_train, network1.train_y)
        Accuracy_epocs_train[e] = A_train

        print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_validation'))
        J_validation = network1.Cost(validation_X_Norm , network1.validation_Y, W, b, lambda_cost)

        J_epocs_validation[e] = J_validation

        P_val, H_val = network1.EvaluationClassifier(layers, validation_X_Norm, W, b)
        k_validation = np.argmax(P_val, axis=0)

        A_validation = network1.ComputeAccuracy(k_validation, network1.validation_y)
        Accuracy_epocs_validation[e] = A_validation
                
        list_temp = [lambda_cost, eta, J_train, J_validation, A_train, A_validation]
        print('params: {} ... time: {}'.format(list_temp, datetime.datetime.now()))
        list_params.append(list_temp)                
        #n_records += 1
        '''

    W = [W1, W2]
    b = [b1, b2]

    #eta_train[e] = eta
    #LambdaCost_train[e] = lambda_cost

    print('\nepoch: {}, batch: {}, t: {}'.format(e, batch, t))
    print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_train'))
    J_train = network1.Cost(train_X_Norm, network1.train_Y, W, b, lambda_cost)

    #J_epocs_train[e] = J_train

    P, H = network1.EvaluationClassifier(layers, train_X_Norm, W, b)
    k_train = np.argmax(P, axis=0)

    A_train = network1.ComputeAccuracy(k_train, network1.train_y)
    #Accuracy_epocs_train[e] = A_train

    print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_validation'))
    J_validation = network1.Cost(validation_X_Norm , network1.validation_Y, W, b, lambda_cost)

    #J_epocs_validation[e] = J_validation

    P_val, H_val = network1.EvaluationClassifier(layers, validation_X_Norm, W, b)
    k_validation = np.argmax(P_val, axis=0)

    A_validation = network1.ComputeAccuracy(k_validation, network1.validation_y)
    #Accuracy_epocs_validation[e] = A_validation
                
    list_temp = [lambda_cost, eta, J_train, J_validation, A_train, A_validation]
    print('params: {} ... time: {}'.format(list_temp, datetime.datetime.now()))
    list_params.append(list_temp)
        
    end_time = datetime.datetime.now()
    
    print("Calculation time of Train_Cyclical: " + str(end_time - start_time))
    print("x-axis (steps) must be multiplied by 10 since the values recorded once in every 10 Cycle")
        
    #print('J_epocs_train: {}'.format(J_epocs_train))
    #print('Accuracy_epocs_train: {}'.format(Accuracy_epocs_train))
    
    #Plot_Train_Validation_Cost_Accurracy(J_epocs_train, J_epocs_validation, Accuracy_epocs_train, Accuracy_epocs_validation)
    
    
    

In [42]:
Train_Cyclical_Coarse_lambda(network1, train_X_Norm, validation_X_Norm, 1, 0.01)

lambda_cost, eta, J_train, J_validation, A_train, A_validation

epoch: 3, batch: 449, t: 1800
time: 2020-05-06 12:07:31.594292 ... operation: J_train
time: 2020-05-06 12:11:10.323563 ... operation: J_validation
params: [0.01, 1e-06, 3.1050863262720654, 3.1092670839080228, 0.10837777777777778, 0.1084] ... time: 2020-05-06 12:11:10.811966
Calculation time of Train_Cyclical: 0:03:41.973654
x-axis (steps) must be multiplied by 10 since the values recorded once in every 10 Cycle


In [ ]:
lambda_c = Coarse_Search(-5, -1)
print(lambda_c)

In [45]:
def Train_Cyclical_Coarse_lambda1(network1, train_X_Norm, validation_X_Norm, n_cycles, lambda_coarse):
    '''
    Up until now, we have trained our networks with Vanilla mini-batch gradient descent. 
    To help speed up training times and avoid time-consuming searches for good values of eta, we will now
    implement mini-batch-GD training where the learning rate at each update step is defined in a cylical way
    check equations (14) and (15) in the assignment.
    '''
    #n_epocs = 200      # number of times we will iterate on the entire data (10K images in our case-1 batch)
    n_batch = 100       # the size of the mini-batch. in other words, number of images in 1 mini-batch.
    #eta = 0.001         # learning rate (step-size)
    lambda_cost = 0.01  # regularization coefficient (punishment)
    d = train_X_Norm.shape[0]
    m = 50 
    K = network1.train_Y.shape[0]  # number of classes
    eta_min = 1e-5
    #eta_max = 1e-1  
    eta_max = 2e-3  # MEL: If I use an eta bigger than 2e-2, I receive an overflow error for SOFTMAX 
    #n_steps = 500                  # number of steps in half a cycle (from eta_min to eta_max) 

    # Generate W1, W2, b1, b2 matrices with initial values
    # (W1, W2, b1, b2) = network1.Initialize_W_b(d, m, K)
    sigma1 = 1 / int(np.sqrt(d))
    sigma2 = 1 / np.sqrt(m)
    #(W1, W2, b1, b2) = network1.Initialize_W_b(d, m, K, sigma1, sigma2)
    initial_sizes = [0, d, m, K]
    (W1, W2, b1, b2) = network1.Initialize_W_b(initial_sizes, sigma1, sigma2)

    W = [W1, W2]
    b = [b1, b2]

    linearLayer1 = layer.Linear()
    reluLayer = layer.ReLU()             # not an exact layer but operational step..
    linearLayer2 = layer.Linear()
    softmaxLayer = layer.Softmax()       # not an exact layer but operational step..
    ReLUlayer = layer.ReLU()
    grad = gradient.Gradient()

    layers = [linearLayer1, reluLayer, linearLayer2, softmaxLayer]
    # total_batch = how many mini-batches will we need to cover the entire training set?
    # we will use 45000 images for training = train_X_Norm.shape[1]
    # total_batch = 45000/100 = 450
    total_batch = int(np.floor(train_X_Norm.shape[1] / n_batch))
    
    # A full cycle once go up (from eta_min to eta_max) and once go down (from eta_max to eta_min)
    # so we multiply by 2
    #n_cycles = 1   # corresponds to "L array" in 2.L.ns in the assignment .. 
    # L=the current cycle number ... n_cycles=the total number of cycles to be applied
    # so L is an element of n_cycle >> i.e. L = {0, 1, 2, 3} if n_cycles = 4 
    # n_epocs=10 for 1 full cycle having n_steps=500 and n_batch=100
    n_steps = 2 * total_batch
    # n_steps / total_batch = 2 is picked before this value was used as k in exercise-3
    # but in exercise-3, we were given the n_steps, now it is calculated and k is given as 2
    # so n_epocs = 2 * 2 * 2 = 8 in our case
    n_epocs = int(2 * n_cycles * (n_steps / total_batch))
    #n_sanity_batch = 1
    # we will only use 100 images for sanity check that means there will be only 1 mini-batch
    #n_sanity_batch = 1
    #n_test_images = n_batch * n_sanity_batch
    
    #J_epocs_train = np.zeros(n_epocs)         # cost array     - will keep costs per epoc (iteration)
    #Accuracy_epocs_train = np.zeros(n_epocs)  # accuracy array - will keep accuracy per epoc (iteration)
    
    n_records = int(2 * n_cycles * n_steps / 10)
    #J_epocs_train = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    #Accuracy_epocs_train = np.zeros(n_records) # we will only record once in 10 iteration
    '''
    J_epocs_train = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_train = np.zeros(n_records) # we will only record once in 10 iteration
    LambdaCost_train = np.zeros(n_records)
    eta_train = np.zeros(n_records)
    
    J_epocs_validation = np.zeros(n_records) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_validation = np.zeros(n_records) # we will only record once in 10 iteration
    '''

    J_epocs_train = np.zeros(n_epocs) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_train = np.zeros(n_epocs) # we will only record once in 10 iteration
    LambdaCost_train = np.zeros(n_epocs)
    eta_train = np.zeros(n_epocs)
    
    J_epocs_validation = np.zeros(n_epocs) # cost array     - will keep costs per epoc (iteration)
    Accuracy_epocs_validation = np.zeros(n_epocs) # we will only record once in 10 iteration

    # t = iteration number for eta (don't confuse with the epocs iteration!)
    t = 0
    # cycleID in use.. if n_cycles=0 then cycle max will be 0. else it will be incremented by 1 at each 2*n_steps
    cycle_no = 0
    n_records = 0
    list_params = []
    start_time = datetime.datetime.now()
    lambda_cost = lambda_coarse
    print('lambda_cost, eta, J_train, J_validation, A_train, A_validation')
    for e in range(n_epocs):
        #lambda_cost = Coarse_Search(-5, -1)
        for batch in range(total_batch):
            #print('cycle_no: {}, e: {}, batch: {}, t: {}, eta: {}'.format(cycle_no, e, batch, t, eta))
            index_list = list(range(batch * n_batch, (batch + 1) * n_batch))
            # shuffling is not necessary but good to have
            np.random.shuffle(index_list)
            #X_batch = train_X_Norm[:, index_list]
            X_batch = train_X_Norm[:, index_list]
            Y_batch = network1.train_Y[:, index_list]

            P, H = network1.EvaluationClassifier(layers, X_batch, W, b)

            G2 = -np.subtract(network1.train_Y[:, index_list], P)  # G
            N2 = Y_batch.shape[1] #N

            # N, G
            (grad_W2, grad_b2, G1) = grad.ComputeGradients_Linear_HiddenLayer(N2, G2, H, lambda_cost, W2) 
            G0 = reluLayer.Backward(G1, H)
            N1 = H.shape[1] #N
            # N, G
            (grad_W1, grad_b1) = grad.ComputeGradients_Linear_FirstLayer(N1, G0, X_batch, lambda_cost, W1)
            
            if 2*cycle_no*n_steps <= t and t <= (2*cycle_no+1)*n_steps:
                eta = eta_min + (t - 2*cycle_no*n_steps)/n_steps*(eta_max-eta_min)
            elif (2*cycle_no+1)*n_steps <= t and t <= 2*(cycle_no+1)*n_steps:
                eta = eta_max - (t - (2*cycle_no+1)*n_steps)/n_steps*(eta_max-eta_min)
            
            # W1star, W2star
            W1 = W1 - eta * grad_W1
            W2 = W2 - eta * grad_W2

            # b1star, b2star
            bstar_m1 = b1 - eta * grad_b1
            b1 = bstar_m1[:, :1]  # there's a broadcast issue needs to be fixed, that's why we pick only 1 column

            bstar_m2 = b2 - eta * grad_b2
            b2 = bstar_m2[:, :1]  # broadcast issue, this is a quick workaround - use 1 column
            
            #print('cycle_no: {}, e: {}, batch: {}, t: {}, eta: {}'.format(cycle_no, e, batch, t, eta))
            
            '''
            # 12:43 if I calculate J for each t
            if t % 10 == 9:    
                W = [W1, W2]
                b = [b1, b2]
                            
                eta_train[n_records] = eta
                LambdaCost_train[n_records] = lambda_cost

                print('\nepoch: {}, batch: {}, n_records: {}, t: {}'.format(e, batch, n_records, t))
                print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_train'))
                J_train = network1.Cost(train_X_Norm, network1.train_Y, W, b, lambda_cost)

                J_epocs_train[n_records] = J_train

                P, H = network1.EvaluationClassifier(layers, train_X_Norm, W, b)
                k_train = np.argmax(P, axis=0)

                A_train = network1.ComputeAccuracy(k_train, network1.train_y)
                Accuracy_epocs_train[n_records] = A_train

                print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_validation'))
                J_validation = network1.Cost(validation_X_Norm , network1.validation_Y, W, b, lambda_cost)

                J_epocs_validation[n_records] = J_validation

                P_val, H_val = network1.EvaluationClassifier(layers, validation_X_Norm, W, b)
                k_validation = np.argmax(P_val, axis=0)

                A_validation = network1.ComputeAccuracy(k_validation, network1.validation_y)
                Accuracy_epocs_validation[n_records] = A_validation
                
                list_temp = [lambda_cost, eta, J_train, J_validation, A_train, A_validation]
                print('params: {} ... time: {}'.format(list_temp, datetime.datetime.now()))
                list_params.append(list_temp)                
                n_records += 1
            '''
            t += 1
            
            if t % (2 * n_steps) == 0:
                cycle_no += 1

        W = [W1, W2]
        b = [b1, b2]

        eta_train[e] = eta
        LambdaCost_train[e] = lambda_cost

        print('\nepoch: {}, batch: {}, t: {}'.format(e, batch, t))
        print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_train'))
        J_train = network1.Cost(train_X_Norm, network1.train_Y, W, b, lambda_cost)

        J_epocs_train[e] = J_train

        P, H = network1.EvaluationClassifier(layers, train_X_Norm, W, b)
        k_train = np.argmax(P, axis=0)

        A_train = network1.ComputeAccuracy(k_train, network1.train_y)
        Accuracy_epocs_train[e] = A_train

        print('time: {} ... operation: {}'.format(datetime.datetime.now(), 'J_validation'))
        J_validation = network1.Cost(validation_X_Norm , network1.validation_Y, W, b, lambda_cost)

        J_epocs_validation[e] = J_validation

        P_val, H_val = network1.EvaluationClassifier(layers, validation_X_Norm, W, b)
        k_validation = np.argmax(P_val, axis=0)

        A_validation = network1.ComputeAccuracy(k_validation, network1.validation_y)
        Accuracy_epocs_validation[e] = A_validation
                
        list_temp = [lambda_cost, eta, J_train, J_validation, A_train, A_validation]
        print('params: {} ... time: {}'.format(list_temp, datetime.datetime.now()))
        list_params.append(list_temp)                
        #n_records += 1
        

    end_time = datetime.datetime.now()
    
    print("Calculation time of Train_Cyclical: " + str(end_time - start_time))
    print("x-axis (steps) must be multiplied by 10 since the values recorded once in every 10 Cycle")
        
    #print('J_epocs_train: {}'.format(J_epocs_train))
    #print('Accuracy_epocs_train: {}'.format(Accuracy_epocs_train))
    
    Plot_Train_Validation_Cost_Accurracy(J_epocs_train, J_epocs_validation, Accuracy_epocs_train, Accuracy_epocs_validation)
    
    
    

In [ ]:
Train_Cyclical_Coarse_lambda1(network1, train_X_Norm, validation_X_Norm, 1, 0.001)

lambda_cost, eta, J_train, J_validation, A_train, A_validation

epoch: 0, batch: 449, t: 450
time: 2020-05-06 14:06:06.893180 ... operation: J_train


In [ ]:
np.sum(k_train == network1.train_y)/k_train.shape[0]

In [ ]:
eta = 0.001
(W2star, b2star, G) =  linearLayer2.Backward(self, N, G, H, lambda_cost, W2, eta, layer_type='hidden')
G = reluLayer.Backward(G)
N = H.shape[1]
(W1star, b1star) =  linearLayer1.Backward(self, N, G, X_batch, lambda_cost, W1, eta, layer_type='first')

In [ ]:
#### TASK - 3 ###
# Top-Level: After reading in and pre-processing the data, you can initialize the parameters of the model 
# W and b as you now know what size they should be. W has size Kxd and b is Kx1. Initialize each entry to have 
# Gaussian random values with zero mean and standard deviation .01. 
# You should use the Matlab function randn to create this data.
asgn1.Task3()